# Configuration

In [1]:
# Parameters
PROJECT_NAME = 'ML1010-Group-Project'
ENABLE_COLAB = False

#Root Machine Learning Directory. Projects appear underneath
GOOGLE_DRIVE_MOUNT = '/content/gdrive' 
COLAB_ROOT_DIR = GOOGLE_DRIVE_MOUNT + '/MyDrive/Colab Notebooks'
COLAB_INIT_DIR = COLAB_ROOT_DIR + '/utility_files'

LOCAL_ROOT_DIR = '/home/magni//ML_Root/project_root'
LOCAL_INIT_DIR = LOCAL_ROOT_DIR + '/utility_files'

# Bootstrap Environment

In [2]:
#add in support for utility file directory and importing
import sys
import os

if ENABLE_COLAB:
  #Need access to drive
  from google.colab import drive
  drive.mount(GOOGLE_DRIVE_MOUNT, force_remount=True)
  
  #add in utility directory to syspath to import 
  INIT_DIR = COLAB_INIT_DIR
  sys.path.append(os.path.abspath(INIT_DIR))
  
  #Config environment variables
  ROOT_DIR = COLAB_ROOT_DIR
  
else:
  #add in utility directory to syspath to import
  INIT_DIR = LOCAL_INIT_DIR
  sys.path.append(os.path.abspath(INIT_DIR))
  
  #Config environment variables
  ROOT_DIR = LOCAL_ROOT_DIR

#Import Utility Support
from jarvis import Jarvis
jarvis = Jarvis(ROOT_DIR, PROJECT_NAME)

import mv_python_utils as mvutils

Wha...where am I?
I am awake now.

I have set your current working directory to /home/magni/ML_Root/project_root/ML1010-Group-Project
The current time is 19:43
Hello sir. I hope you had dinner.



# Setup Runtime Environment

In [3]:
if ENABLE_COLAB:
  #!pip install scipy -q
  #!pip install scikit-learn -q
  #!pip install pycaret -q
  #!pip install matplotlib -q
  #!pip install joblib -q
  #!pip install pandasql -q
  !pip install umap_learn -q
  !pip install sentence_transformers -q
  !pip install spacytextblob -q
  !pip install flair -q
  display('Google Colab enabled')
else:
  display('Google Colab not enabled')

#Common imports
import json
import pandas as pd
import numpy as np
import matplotlib
import re
import nltk
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split as tts
#from yellowbrick.classifier import ConfusionMatrix
#from sklearn.linear_model import LogisticRegression
from yellowbrick.target import ClassBalance
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix


nltk.download('stopwords')
%matplotlib inline

'Google Colab not enabled'

[nltk_data] Downloading package stopwords to /home/magni/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import cw_df_metric_utils as cwutils

2022-01-08 19:43:26.045625: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-08 19:43:26.045656: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
axis_labels5=[1,2,3,4,5]
axis_labels2=[0,1]

# Load Data

In [ ]:
jarvis.getPackageVersion('pandas')
!python -V
jarvis.showProjectDataFiles()

In [ ]:
dataSrc = pd.read_pickle(jarvis.DATA_DIR_WORK + "/01_NL_ReviewText_All(new).pkl.gz")
mvutils.exploreDataframe (dataSrc, numRecords=1)

# Lemma - dropNA and Balance (2 class and 5 class)

In [ ]:
dataCoreLemma = dataSrc[['uuid',
                         'overall',
                         'overall_posneg',
                         'reviewText_lemma',
                         'reviewText_lemma_tb_pol',
                         'reviewText_lemma_tb_subj',
                         'reviewText_lemma_tb_tokens', 
                         'reviewText_lemma_tb_length',
                         'reviewText_lemma_bert',
                         'reviewText_lemma_flairSent',
                         'reviewText_lemma_flairSent_norm',
                         'reviewText_lemma_flairSent_posneg',
                         'reviewText_lemma_tb_pol_norm',
                         'reviewText_lemma_tb_pol_posneg'
                        ]].copy()

In [ ]:
dataCoreLemma.info()

In [ ]:
#Drop null values from flairSent
dataCoreLemma.dropna(subset=['reviewText_lemma_flairSent'], inplace=True)
dataCoreLemma.reset_index(drop=True, inplace=True)
dataCoreLemma.info()

In [ ]:
dataCoreLemmaBal2 = mvutils.classBalanceUndersample(dataCoreLemma, 'overall_posneg')

In [ ]:
dataCoreLemmaBal5 = mvutils.classBalanceUndersample(dataCoreLemma, 'overall')

# Lemma - Prune data and Balance (2 class and 5 class)

In [ ]:
dataCoreLemmaPrune = dataCoreLemma.copy()
dataCoreLemmaPrune.info()

In [ ]:
mvutils.examineColumnNumeric(dataCoreLemmaPrune, 
                             'reviewText_lemma_tb_tokens', 
                             binsize=1, 
                             zoom=True,
                             minZoomLevel=0,
                             maxZoomLevel=20,
                             plotsize=5)

In [ ]:
mvutils.examineColumnNumeric(dataCoreLemmaPrune, 
                             'reviewText_lemma_tb_tokens', 
                             binsize=100, 
                             zoom=True,
                             minZoomLevel=1500,
                             maxZoomLevel=5000,
                             plotsize=5)

In [ ]:
#import numpy as np
idx = np.where((dataCoreLemmaPrune['reviewText_lemma_tb_tokens']>=5) &
               (dataCoreLemmaPrune['reviewText_lemma_tb_tokens']<= 2100))

In [ ]:
dataCoreLemmaPrune = dataCoreLemmaPrune.loc[idx].copy()
dataCoreLemmaPrune.reset_index(inplace=True, drop=True)

In [ ]:
dataCoreLemmaPrune.info()

In [ ]:
dataCoreLemmaPruneBal2 = mvutils.classBalanceUndersample(dataCoreLemmaPrune, 'overall_posneg')

In [ ]:
dataCoreLemmaPruneBal5 = mvutils.classBalanceUndersample(dataCoreLemmaPrune, 'overall')

**Reporting [Lemma.Core.Flair.5 Star]**

In [ ]:
cwutils.showTestReport(df=dataCoreLemmaBal5,
               colNameActual='overall',
               colNamePredict='reviewText_lemma_flairSent_norm',
               axisLabels=axis_labels5,
               chartTitle='Flair [Lemma.Core.5]')

**Reporting [Lemma.Core.Flair.Pos/Neg]**

In [ ]:
cwutils.showTestReport(df=dataCoreLemmaBal2,
               colNameActual='overall_posneg',
               colNamePredict='reviewText_lemma_flairSent_posneg',
               axisLabels=axis_labels2,
               chartTitle='Flair [Lemma.Core.PosNeg]')

**Reporting [Lemma.Core.spaCyTextBlob.5 Star]**

In [ ]:
cwutils.showTestReport(df=dataCoreLemmaBal5,
               colNameActual='overall',
               colNamePredict='reviewText_lemma_tb_pol_norm',
               axisLabels=axis_labels5,
               chartTitle='spaCy/TextBlob [Lemma.Core.5]')

**Reporting [Lemma.Core.SpacyTextBlob.Pos/Neg]**

In [ ]:
cwutils.showTestReport(df=dataCoreLemmaBal2,
               colNameActual='overall_posneg',
               colNamePredict='reviewText_lemma_tb_pol_posneg',
               axisLabels=axis_labels2,
               chartTitle='spaCy/TextBlob [Lemma.Core.PosNeg]')

# BERT - [Lemma.Core]

In [ ]:
import importlib
importlib.reload(cwutils)


In [ ]:
classifier = XGBClassifier(eval_metric='mlogloss')

modelBertCoreLemma5, dfmodelBertCoreLemma5 = cwutils.createBertModel(df=dataCoreLemmaBal5,
                bertColumn='reviewText_lemma_bert',
                uniqueColumn='uuid',
                targetColumn='overall',
                classifier=classifier
                )



In [ ]:
cwutils.showTestReport(df=dfmodelBertCoreLemma5,
               colNameActual='y_test',
               colNamePredict='y_pred',
               axisLabels=axis_labels5,
               chartTitle='BERT [Lemma.Core.5]')

In [ ]:
import importlib
importlib.reload(cwutils)


In [ ]:
tDf = cwutils.plotModelFeatureImportance(modelBertCoreLemma5, returnAbove=0.0020)

modelTest, dfTestModel= cwutils.createBertModel(df=dataCoreLemmaBal5,
                bertColumn='reviewText_lemma_bert',
                uniqueColumn='uuid',
                targetColumn='overall',
                classifier=classifier,
                featureFilter=tDf['feature'].tolist()                                                     
                )

cwutils.showTestReport(df=dfTestModel,
               colNameActual='y_test',
               colNamePredict='y_pred',
               axisLabels=axis_labels5,
               chartTitle='BERT [Lemma.Core.5]')

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold


dfBert = cwutils.getBertEncodeFrame(df=dataCoreLemmaBal5,
                                    bertColumn='reviewText_lemma_bert',
                                    uniqueColumn='uuid',
                                    otherColumns=['overall']
                                    )

#Get list of all columns
features = list(dfBert.columns)

#Remove unique and target colums
features.remove('uuid')
features.remove('overall')
features_all = features.copy()
features_small = features.copy()

#Get list of "low value" features, the "filter"
featureFilter = tDf['feature'].tolist()

#Remove high first so list shortens from tail
featureFilter.sort(reverse=True)
#Remove features identified previously
#So we only have the good ones left
for x in featureFilter:
    del features_small[x]

print (len(features_all))
print (len(features_small))
print (len(featureFilter))

cv = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
target = 'overall'


fig, axes = plt.subplots(3, 2, figsize=(10, 15))

title = "Learning Curves (Naive Bayes)"

#xgb_model = xgboost.XGBClassifier(num_class=7,
#                                  learning_rate=0.1,
#                                  num_iterations=1000,
#                                  max_depth=10,
#                                  feature_fraction=0.7, 
#                                  scale_pos_weight=1.5,
#                                  boosting='gbdt',
#                                  metric='multiclass',
#                                  eval_metric='mlogloss'
                                
estimator = XGBClassifier(eval_metric='mlogloss')
plot_learning_curve(
    estimator, title, dfBert[features_small], dfBert[target], axes=axes[:, 0], ylim=(0.0, 1.01), cv=cv, n_jobs=16
)

title = r"Learning Curves (SVM, RBF kernel, $\gamma=0.001$)"
estimator = SVC(gamma=0.001)
plot_learning_curve(
    estimator, title, dfBert[features_small], dfBert[target], axes=axes[:, 1], ylim=(0.0, 1.01), cv=cv, n_jobs=16
)

plt.show()


# Scratchpad

In [6]:
import importlib
import DataPackage as dp

In [ ]:
importlib.reload(dp)

In [7]:
testDf = pd.read_pickle(jarvis.DATA_DIR_WORK + "/01_NL_ReviewText_All(new).pkl.gz")
testFilename = '01_alphaTest'
#mvutils.exploreDataframe (testDf, numRecords=1)
testDfBert = cwutils.getBertEncodeFrame(df=testDf,
                                        bertColumn='reviewText_lemma_bert',
                                        uniqueColumn='uuid',
                                        otherColumns=['overall']
                                        )

In [8]:
myDataPackage = dp.DataPackage(testDfBert)

In [9]:
jarvis.saveExperiment(myDataPackage, testFilename)

In [10]:
loadedDataPackage = jarvis.loadExperiment(testFilename)

In [11]:
mvutils.exploreDataframe (loadedDataPackage.getDataFrame(), numRecords=1)

dataframe shape: (63413, 770)

dataframe info: 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 63413 entries, 0 to 63412
Columns: 770 entries, uuid to overall
dtypes: float32(768), float64(1), object(1)
memory usage: 187.2+ MB
None

Top 1 in dataframe


,uuid,c0,c1,c2,c3,c4,c5,c6,c7,c8,...,c759,c760,c761,c762,c763,c764,c765,c766,c767,overall
0,e5322688-1105-401b-be69-888bc1d89bcf,0.472575,1.131676,0.477244,0.40458,1.095912,0.094882,1.815453,0.733118,1.012068,...,0.241637,-0.54165,-0.52954,0.28598,0.032169,0.533272,0.211305,1.009963,1.077491,1.0



Bottom 1 in dataframe


,uuid,c0,c1,c2,c3,c4,c5,c6,c7,c8,...,c759,c760,c761,c762,c763,c764,c765,c766,c767,overall
63412,8f71ec3b-73e0-408e-b7e0-acd146f697e7,-0.114804,0.812048,0.726903,0.353847,1.201606,0.093721,1.054865,0.254875,1.106876,...,-1.019994,-0.142057,-0.613902,0.36779,0.136367,0.085395,0.539971,0.42914,0.903761,4.0
